모델의 아키텍처는 총 3개의 층으로 구성된다.

1번 레이어: 합성곱층<br>
합성곱(in_channel=1, out_channel=32, kernel_size=3, stride=1, padding=1) + ReLU<br>
맥스풀링(kernel_size=2, stride=2)

2번 레이어: 합성곱층<br>
합성곱(in_channel=32, out_channel=64, kernel_size=3, stride=1, padding=1) + ReLU<br>
맥스풀링(kernel_size=2, stride=2)

3번 레이어: 전결합층(Fully-Connected Layer)<br>
특성맵을 펼친다.<br>
전결합층(뉴런 10개) + Softmax

In [1]:
import torch
import torch.nn as nn
import torch.nn.init
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
torch.manual_seed(7777)
if device == "cuda":
    torch.cuda.manual_seed_all(777)

In [4]:
lr = 1e-3
training_epochs = 15
batch_size = 100

In [5]:
mnist_train = dsets.MNIST(
    root="../input/", train=True, transform=transforms.ToTensor(), download=True
)
mnist_test = dsets.MNIST(
    root="../input/", train=False, transform=transforms.ToTensor(), download=True
)

In [6]:
train_loader = torch.utils.data.DataLoader(
    dataset=mnist_train, batch_size=batch_size, shuffle=True, drop_last=True
)
test_loader = torch.utils.data.DataLoader(
    dataset=mnist_test, batch_size=batch_size, shuffle=False
)

In [7]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.fc = nn.Linear(7 * 7 * 64, 10, bias=True)

        # 전결합층 한정 가중치 초기화
        nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        # 배치 차원만 남기고 Flatten
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [8]:
model = CNN().to(device)

In [9]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [10]:
total_batch = len(train_loader)
total_batch

600

In [11]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, y in train_loader:
        X = X.to(device)
        y = y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print(f"[Epoch {epoch + 1:>4}] cost = {avg_cost:>.9}")

[Epoch    1] cost = 0.222355321
[Epoch    2] cost = 0.0612181798
[Epoch    3] cost = 0.0432155654
[Epoch    4] cost = 0.0353079364
[Epoch    5] cost = 0.0299282055
[Epoch    6] cost = 0.0238759629
[Epoch    7] cost = 0.0209481567
[Epoch    8] cost = 0.017400112
[Epoch    9] cost = 0.0138331493
[Epoch   10] cost = 0.0120316716
[Epoch   11] cost = 0.00995096005
[Epoch   12] cost = 0.00908031315
[Epoch   13] cost = 0.00717559271
[Epoch   14] cost = 0.0073594735
[Epoch   15] cost = 0.00544247311


In [12]:
with torch.no_grad():
    accuracy = 0
    for X, y in test_loader:
        X = X.to(device)
        y = y.to(device)

        prediction = model(X)
        correct_prediction = torch.argmax(prediction, dim=1) == y

        accuracy += correct_prediction.float().sum()
    accuracy /= len(mnist_test)
    print(f"Accuracy: {accuracy.item()}")

Accuracy: 0.9896000027656555
